# Infor722 Iteration 4 

In [25]:
import findspark
findspark.init('/home/ubuntu/spark-3.2.1-bin-hadoop2.7')
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, count, when

In [3]:
spark = SparkSession.builder.master("local[1]").appName("SparkApp")\
    .config("spark.executor.memory", "4g")\
    .config("spark.driver.memory", "4g") \
    .getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/05/16 07:18:18 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


## Data Understand

### Load Dataset

In [15]:
df_main = spark.read.csv('Datasets/diabetes_012_health_indicators_BRFSS2021.csv', header=True)
df_income = spark.read.csv('Datasets/diabetes_Income.csv', header=True)

### Explore Data

In [16]:
df_desc = df_main.describe()
df_desc.show()

+-------+-------------------+------------------+-------------------+-------------------+------------------+-------------------+-------------------+--------------------+------------------+-------------------+-------------------+-------------------+-------------------+-------------------+------------------+-----------------+-----------------+------------------+------+-----------------+------------------+
|summary|       Diabetes_012|            HighBP|           HighChol|          CholCheck|               BMI|             Smoker|             Stroke|HeartDiseaseorAttack|      PhysActivity|             Fruits|            Veggies|  HvyAlcoholConsump|      AnyHealthcare|        NoDocbcCost|           GenHlth|         MentHlth|         PhysHlth|          DiffWalk|   Sex|              Age|         Education|
+-------+-------------------+------------------+-------------------+-------------------+------------------+-------------------+-------------------+--------------------+------------------+-

In [18]:
df_main.printSchema()

root
 |-- Diabetes_012: string (nullable = true)
 |-- HighBP: string (nullable = true)
 |-- HighChol: string (nullable = true)
 |-- CholCheck: string (nullable = true)
 |-- BMI: string (nullable = true)
 |-- Smoker: string (nullable = true)
 |-- Stroke: string (nullable = true)
 |-- HeartDiseaseorAttack: string (nullable = true)
 |-- PhysActivity: string (nullable = true)
 |-- Fruits: string (nullable = true)
 |-- Veggies: string (nullable = true)
 |-- HvyAlcoholConsump: string (nullable = true)
 |-- AnyHealthcare: string (nullable = true)
 |-- NoDocbcCost: string (nullable = true)
 |-- GenHlth: string (nullable = true)
 |-- MentHlth: string (nullable = true)
 |-- PhysHlth: string (nullable = true)
 |-- DiffWalk: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: string (nullable = true)
 |-- Education: string (nullable = true)



In [19]:
diabetes_counts = df_main.groupBy("Diabetes_012").count()
diabetes_counts.show()

+------------+------+
|Diabetes_012| count|
+------------+------+
|           0|197191|
|           1|  5619|
|           2| 33568|
+------------+------+



### Verify the data quality

In [23]:
df_main.printSchema()

root
 |-- Diabetes_012: string (nullable = true)
 |-- HighBP: string (nullable = true)
 |-- HighChol: string (nullable = true)
 |-- CholCheck: string (nullable = true)
 |-- BMI: string (nullable = true)
 |-- Smoker: string (nullable = true)
 |-- Stroke: string (nullable = true)
 |-- HeartDiseaseorAttack: string (nullable = true)
 |-- PhysActivity: string (nullable = true)
 |-- Fruits: string (nullable = true)
 |-- Veggies: string (nullable = true)
 |-- HvyAlcoholConsump: string (nullable = true)
 |-- AnyHealthcare: string (nullable = true)
 |-- NoDocbcCost: string (nullable = true)
 |-- GenHlth: string (nullable = true)
 |-- MentHlth: string (nullable = true)
 |-- PhysHlth: string (nullable = true)
 |-- DiffWalk: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: string (nullable = true)
 |-- Education: string (nullable = true)



In [29]:
# Count the number of missing values per column
missing_values = df_main.select([count(when(col(c).isNull(), c)).alias(c) for c in df_main.columns])
missing_values.show()

+------------+------+--------+---------+---+------+------+--------------------+------------+------+-------+-----------------+-------------+-----------+-------+--------+--------+--------+---+---+---------+
|Diabetes_012|HighBP|HighChol|CholCheck|BMI|Smoker|Stroke|HeartDiseaseorAttack|PhysActivity|Fruits|Veggies|HvyAlcoholConsump|AnyHealthcare|NoDocbcCost|GenHlth|MentHlth|PhysHlth|DiffWalk|Sex|Age|Education|
+------------+------+--------+---------+---+------+------+--------------------+------------+------+-------+-----------------+-------------+-----------+-------+--------+--------+--------+---+---+---------+
|           0|     0|       1|        1| 14|     9|     7|                  10|          12|     9|      7|                9|            7|          4|      5|       3|       6|       4|  0|  1|        2|
+------------+------+--------+---------+---+------+------+--------------------+------------+------+-------+-----------------+-------------+-----------+-------+--------+--------+---

In [27]:
# Percentage of missing values in the dataset
total_rows = df_main.count()
missing_percentage = df_main.select([(count(when(col(c).isNull(), c)) / total_rows * 100).alias(c) for c in df_main.columns])
missing_percentage.show()

+------------+------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+---+--------------------+--------------------+
|Diabetes_012|HighBP|            HighChol|           CholCheck|                 BMI|              Smoker|              Stroke|HeartDiseaseorAttack|        PhysActivity|              Fruits|             Veggies|   HvyAlcoholConsump|       AnyHealthcare|         NoDocbcCost|             GenHlth|            MentHlth|            PhysHlth|            DiffWalk|Sex|                 Age|           Education|
+------------+------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+---

In [28]:
# Value counts for the 'Sex' column
sex_distribution = df_main.groupBy("Sex").count()
sex_distribution.show()

+------+------+
|   Sex| count|
+------+------+
|     F|     3|
|Female|123426|
|female|     2|
|     M|     3|
|  feee|     1|
|  Male|112943|
+------+------+



## Data preparation